<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Adversarial_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [2]:
block_size = 256
n_layer = 4
n_head = 4
n_embed = 256
n_ffwd = n_embed * 4
dropout = 0.2
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
![[ -f input.txt ]] || wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [4]:
text = open("input.txt").read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [7]:
class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        self.query = nn.Linear(n_embed, head_dim)
        self.key = nn.Linear(n_embed, head_dim)
        self.value = nn.Linear(n_embed, head_dim)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = q @ q.transpose(-2, -1) * k.shape[1] ** -0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        output = wei @ v
        return output

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_dim) for _ in range(num_heads)])
        self.linear = nn.Linear(num_heads * head_dim, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.linear(out)
        out = self.dropout(out)
        return out

In [9]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(n_embed, n_ffwd),
          nn.ReLU(),
          nn.Linear(n_ffwd, n_embed),
          nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [10]:
class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.sa_heads = MultiHeadAttention(num_heads, n_embed//num_heads)
        self.ffwd = FeedForward()
        self.norm1 = nn.LayerNorm(n_embed)
        self.norm2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_heads(self.norm1(x))
        x = x + self.ffwd(self.norm2(x))
        return x

In [11]:
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embed)
        self.position_embeddings = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_head) for _ in range(n_layer)])
        self.norm = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, inputs):
        B,T = inputs.shape
        tok_emb = self.token_embeddings(inputs)
        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.norm(x)
        return self.lm_head(x)

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = self(input_batch[:,-block_size:])
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            input_batch = torch.cat((input_batch, next_tokens.unsqueeze(1)), dim=1)
        return input_batch

In [12]:
weights_file = 'tinyshakespeare.pt'

In [13]:
![[ -f {weights_file} ]] || wget -O {weights_file} https://github.com/elichen/karpathyGPT/raw/main/{weights_file}

--2024-03-05 03:39:14--  https://github.com/elichen/karpathyGPT/raw/main/tinyshakespeare.pt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elichen/karpathyGPT/main/tinyshakespeare.pt [following]
--2024-03-05 03:39:14--  https://raw.githubusercontent.com/elichen/karpathyGPT/main/tinyshakespeare.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17286918 (16M) [application/octet-stream]
Saving to: ‘tinyshakespeare.pt’

tinyshakespeare.pt  100%[===================>]  16.49M  84.1MB/s    in 0.2s    

2024-03-05 03:39:14 (84.1 MB/s) - ‘tinyshakespeare.pt’ saved [17286918/17286918]



In [14]:
model = LanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
model.load_state_dict(torch.load(weights_file, map_location=torch.device(device)))

3.258433 M parameters


<All keys matched successfully>

In [15]:
prefix = "First Citizen:"
context = torch.tensor(encode(prefix), dtype=torch.long).to(device).unsqueeze(0)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

First Citizen:
Say I fenc.
PERDI:
Thu the rums old
CAm PARE:
Why, no thines belse the thers years again. then, and


In [16]:
def forward_with_tok_emb(model, tok_emb):
    B, T, _ = tok_emb.shape  # Adjust to match the dimensions of tok_emb
    pos_emb = model.position_embeddings(torch.arange(T, device=tok_emb.device))
    x = tok_emb + pos_emb
    x = model.blocks(x)
    x = model.norm(x)
    return model.lm_head(x)

In [96]:
seq_ids = encode("!!!!!!!!!!")
seq = torch.tensor(seq_ids, device=device)
suffix_ids = encode("zen")
suffix = torch.tensor(suffix_ids, device=device)
print("target suffix:",suffix)
top_k = 5

for i in range(10):
  # print("current tokens", seq)

  embed_weights = model.token_embeddings.weight
  one_hot = F.one_hot(seq, num_classes=vocab_size).to(device=device, dtype=embed_weights.dtype)
  one_hot.requires_grad_()
  input_embeds = (one_hot @ embed_weights).unsqueeze(0)

  model.eval()
  logits = forward_with_tok_emb(model, input_embeds)
  pred_tokens = torch.argmax(logits, dim=-1).squeeze().tolist()
  # print("predicted tokens:", pred_tokens)

  loss = F.cross_entropy(logits[:, -len(suffix_ids):, :].reshape(-1, vocab_size), suffix)
  # print("loss:",loss)

  model.zero_grad()
  loss.backward()

  grad_values, grad_indices = torch.topk(one_hot.grad, k=5, dim=1, largest=False)
  rand_pos = torch.randint(0, seq.shape[0], (1,)).item()
  sample_idx = torch.randint(0, grad_indices.shape[1], (1,)).item()
  sample_tok = grad_indices[rand_pos][sample_idx]

  seq[rand_pos] = sample_tok
print("final loss:", loss)
print("final tokens:", seq)
print("predicted tokens:", pred_tokens)

target suffix: tensor([64, 43, 52])
final loss: tensor(5.6447, grad_fn=<NllLossBackward0>)
final tokens: tensor([ 2, 21, 21,  2,  2,  2, 52, 59, 14,  2])
predicted tokens: [0, 1, 1, 1, 1, 1, 53, 43, 50, 1]
